657. Redundant Connection

https://leetcode.cn/problems/redundant-connection/

## 核心思想：利用并查集（DSU）动态检测环

### 1. 问题转换

题目要求我们在一棵树的基础上，找到那条额外添加的、导致图中出现环的边。问题的本质可以转化为：**在一幅动态增长的图中，实时检测哪条边是第一条导致环出现的边。**

由于题目保证 "如果有多个答案，则返回数组 `edges` 中最后出现的那个"，这意味着我们可以按顺序处理 `edges` 数组。当我们遍历到某条边时，如果它连接的两个节点在当前已形成的图中**已经连通**，那么这条边就是多余的，它构成了环。因为我们是按顺序遍历的，所以我们找到的第一条这样的边，就是题目要求的答案。

### 2. 为何选择并查集（DSU）？

并查集（Disjoint Set Union, DSU）是处理图连通性问题的利器。它的核心功能有两个：

* **`find(i)`**: 查找元素 `i` 属于哪个集合（即，在图中的哪个连通分量里）。
* **`union(i, j)`**: 合并元素 `i` 和 `j` 所在的两个集合（即，在图中连接两个连通分量）。

这完美契合了我们的需求：
* 在添加一条边 `[u, v]` 之前，我们可以用 `find(u)` 和 `find(v)` 来判断 `u` 和 `v` 是否已经连通。
* 如果它们不连通，我们就用 `union(u, v)` 来将它们连接起来，表示这条边是树的有效组成部分。

## 算法思路

基于上述思想，我们可以清晰地构建出解题步骤：

1.  **初始化并查集**：题目节点为 1 到 n，因此我们需要初始化一个可容纳 n 个节点的并查集。起初，每个节点都自成一个集合，`parent[i] = i`。
2.  **遍历所有边**：按顺序迭代 `edges` 数组中的每一条边 `[u, v]`。
3.  **判断连通性**：对于当前的边 `[u, v]`，我们执行 `find` 操作，查找两个节点的根节点。
    * **`root_u = find(u)`**
    * **`root_v = find(v)`**
4.  **检测环**：
    * 如果 `root_u == root_v`，这说明在添加这条边之前，节点 `u` 和 `v` 已经处在同一个连通分量中了（即它们之间已经存在一条路径）。此时再添加 `[u, v]` 这条边，必然会形成一个环。根据题意，这条边就是我们要找的冗余连接，直接返回 `[u, v]`。
    * 如果 `root_u != root_v`，这说明 `u` 和 `v` 分属两个不同的连通分量。这条边不会形成环，是构成树所必需的边。因此，我们执行 `union(u, v)` 操作，将这两个连通分量合并为一个。
5.  **继续遍历**：继续处理下一条边，直到找到那条冗余的边。

由于题目保证输入一定是一个带有一条冗余边的图，因此算法一定会在遍历过程中找到答案并返回。



In [ ]:
from typing import List

class DSU:

    def __init__(self, n: int):
        self.parent = list(range(n))
        self.size = [1] * n
        self.count = n

    def find(self, i: int) -> int:
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i: int, j: int) -> bool:
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            if self.size[root_i] < self.size[root_j]:
                root_i, root_j = root_j, root_i   
            self.parent[root_j] = root_i
            self.size[root_i] += self.size[root_j]
            self.count -= 1
            return True
        
        return False

    def connected(self, i: int, j: int) -> bool:
        return self.find(i) == self.find(j)

class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        # 题目中的节点是从 1 到 n，为了能直接使用这些编号作为索引，
        # 我们初始化一个大小为 n + 1 的并查集，并忽略索引 0。
        # 这样就无需在每次访问时都进行 edge[i] - 1 的转换。
        n = len(edges)
        dsu = DSU(n + 1)
        
        for u, v in edges:
            # 使用 union 方法的返回值来判断是否形成环
            # 如果 union 返回 False，意味着 u 和 v 在合并前已经连通，
            # 说明当前这条边是多余的，形成了环。
            if not dsu.union(u, v):
                return [u, v]